In [ ]:
# Cell 1 — Imports, Tracing & Environment

# Standard boilerplate — used in every agents notebook
# This is the standard opening cell for every notebook that uses the Agents SDK. Import, disable tracing, load .env, configure OpenRouter, set the model. Always in this order.
import os
from dotenv import load_dotenv

# Agent  — describes one AI agent: its name, instructions, and model.
#           Does nothing alone — Runner executes it.
# Runner — the engine that sends the agent's message to the AI model
#           and returns the result.
# set_tracing_disabled — silences 401 trace errors from OpenRouter usage.
from agents import Agent, Runner, set_tracing_disabled

# Disable tracing FIRST — before any agent is created or run
set_tracing_disabled(True)

load_dotenv(override=True)

# Point the Agents SDK at OpenRouter instead of OpenAI
os.environ["OPENAI_API_KEY"]  = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

# Read model from .env — second argument is the fallback default
MODEL = os.getenv("MODEL", "openai/gpt-4o-mini")

print("✅ Environment ready. Tracing disabled.")
print(f"   Model: {MODEL}")

In [ ]:
# Cell 2 — Define the Test Code Snippet

# Triple quotes (""") let you write a string spanning multiple lines. This is how we store a block of code as a Python variable to send to the agent.

test_code = """
def calculate_discount(price: float, discount_percent: float) -> float:
    if discount_percent < 0 or discount_percent > 100:
        raise ValueError("Discount must be between 0 and 100")
    discount_amount = price * (discount_percent / 100)
    return price - discount_amount

def apply_bulk_discount(prices: list, threshold: int, discount: float) -> list:
    if len(prices) >= threshold:
        return [calculate_discount(p, discount) for p in prices]
    return prices
"""

print("Test code defined.")
print(test_code)

In [ ]:
# Cell 3 — Create the Code Analyst Agent

# Agent() takes three things: a name (label only, no effect on AI behaviour), instructions (the system prompt — this is what shapes every response), and a model. A precise system prompt produces reliable, structured output every time. A vague one produces vague, inconsistent output.

code_analyst = Agent(
    name="Code Analyst",
    instructions="""
    You are an expert code analyst. Your job is to read Python or
    JavaScript code and produce a clear, structured analysis.

    For every function and class you find, identify and report:
    - The name
    - What it does in plain English (one or two sentences)
    - All parameters: their names and what they represent
    - The return value: what it returns and what type it is
    - Any edge cases, error conditions, or assumptions baked in
    - Any dependencies on other functions in the code

    Format your output clearly with sections and bullet points.
    Be precise and technical. Do not add padding or filler sentences.
    Your output will be passed directly to a documentation writer agent,
    so completeness and accuracy are critical.
    """,
    model=MODEL
)

print(f"✅ Agent created: {code_analyst.name}")

In [6]:
# Cell 4 — Run the Agent with

# await Runner.run()
# Execute the agent — Jupyter-correct async call

# We use await Runner.run() — not run_sync(). Jupyter already has an event loop running. run_sync() tries to create a second one and Python refuses. await hands the work to the existing loop instead.

# result.final_output is a plain string — the agent's text response. This is what gets passed to the next agent in the full pipeline.

user_message = f"""Please analyse the following Python code and
provide a complete structured analysis:

{test_code}
"""

print("Running Code Analyst agent...")
print("-" * 60)

# await Runner.run() — correct for Jupyter.
# Works with Jupyter's existing event loop.
# Never use run_sync() in a notebook.
result = await Runner.run(code_analyst, user_message)

# result.final_output — the agent's text response as a plain string
analysis = result.final_output
print(analysis)

# Look for
# Both functions named · plain-English descriptions · parameters with types · ValueError edge case · dependency between functions noted

Running Code Analyst agent...
------------------------------------------------------------
# Code Analysis

## Function: `calculate_discount`

- **Name**: `calculate_discount`
- **Description**: Calculates the final price after applying a discount percentage to the original price.
- **Parameters**:
  - `price` (float): The original price before the discount is applied.
  - `discount_percent` (float): The percentage of discount to apply to the original price.
- **Return value**: Returns the final price after discount is applied, of type `float`.
- **Edge Cases/Error Conditions/Assumptions**:
  - Raises a `ValueError` if `discount_percent` is less than 0 or greater than 100, ensuring valid percentage input.
- **Dependencies**: None.

---

## Function: `apply_bulk_discount`

- **Name**: `apply_bulk_discount`
- **Description**: Applies a bulk discount to a list of prices if the number of prices meets or exceeds a specified threshold.
- **Parameters**:
  - `prices` (list): A list of prices 

In [7]:
# Cell 5 — Inspect the Result Object

# See what the result object contains

# The result holds more than just final_output. Understanding its structure helps when debugging in later phases.

print("Result object details:")
print(f"  Type:                {type(result)}")
print(f"  final_output type:   {type(result.final_output)}")
print(f"  final_output length: {len(result.final_output)} characters")

# dir() lists all attributes on an object.
# We filter out names starting with '_' — those are internal Python things.
public_attrs = [attr for attr in dir(result) if not attr.startswith('_')]
print(f"  Available attributes: {public_attrs}")

Result object details:
  Type:                <class 'agents.result.RunResult'>
  final_output type:   <class 'str'>
  final_output length: 1467 characters
  Available attributes: ['context_wrapper', 'final_output', 'final_output_as', 'input', 'input_guardrail_results', 'interruptions', 'last_agent', 'last_response_id', 'max_turns', 'new_items', 'output_guardrail_results', 'raw_responses', 'release_agents', 'to_input_list', 'to_state', 'tool_input_guardrail_results', 'tool_output_guardrail_results']


In [8]:
# # ✅ Notebook 02 Complete
# # Created an Agent with a focused system prompt
# # Ran it correctly in Jupyter with await Runner.run()
# # Read result.final_output — the string we pass forward
# # Understood why run_sync() never goes in a notebook
# → Next: 03_multi_agent_handoff_pipeline.ipynb